# MrPUP-5: the patient counts for each pair of NPI in HOP Teaming should be higher than patient count of same NPI pairs in MrPuP v2

QA step description: check if all the patient counts for each pair of NPI in HOP Teaming are higher than patient count of same NPI pairs in MrPuP v2. 

Starting Author: Amy Jin (amy@careset.com)

Date: July 16th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connect to Parenthood server:

In [5]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [12]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_hop:                  hop teaming database name in server
# 2）table_hop:               hop teaming table name
# 3) npi:                     npi in hop teaming table
# 4) npi_dest:                npi_dest in hop teaming table
# 5) hop_col:                 patient count in hop teaming table

# 1) db_mrpup:                  mrpup database name in server
# 2）table_mrpup:               mrpup table name
# 3) rfr_physn_npi:             referring npi in mrpup table
# 4) prf_physn_npi:             performing npi in mrpup table
# 5) mrpup_col:                 patient count in mrpup table
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, the test will print out duplicate (rfr_physn_npi, HCPCS, carr_clm_blg_npi_num).


def mrpup_5(db_hop, table_hop, npi, npi_dest, hop_col,  \
            db_mrpup, table_mrpup, rfr_physn_npi, prf_physn_npi, mrpup_col):
    #table1 = str(db_name) + '.' + str(table_name)
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
       
        # MySQL to get failed rows
        query = ('''
                SELECT hop.{col1} AS hop_bene_cnt, 
                       V2.cnt_bene_id_sum AS mrpup2_bene_cnt,
                       {rfr_physn_npi},
                       {prf_physn_npi}
                FROM {db1}.{t1} AS hop
                JOIN (SELECT
                        SUM(mrpup.{col2}) AS cnt_bene_id_sum,
                        mrpup.{rfr_physn_npi},
                        mrpup.{prf_physn_npi}
                      FROM {db2}.{t2} AS mrpup
                      GROUP BY mrpup.{prf_physn_npi}, mrpup.{rfr_physn_npi}
                      ) AS V2
                ON hop.{npi} = V2.{rfr_physn_npi} AND hop.{npi_dest} = V2.{prf_physn_npi}
                WHERE hop.{col1} < V2.cnt_bene_id_sum; 
        '''.format(db1 = db_hop, db2 = db_mrpup, \
                    t1 = table_hop, t2 = table_mrpup, \
                    npi = npi, npi_dest = npi_dest, \
                    rfr_physn_npi = rfr_physn_npi, prf_physn_npi = prf_physn_npi,\
                    col1 = hop_col, col2 = mrpup_col))

        cur.execute(query, params=None, multi=True)


        for result in cur.execute(query, multi=True):
            result.fetchall()                         
            print("Test Mrpup-5 statement: '{}':".format(result.statement)+'\n')   
            print("Number of rows fail the test is: {}".format(result.rowcount) +'\n')

        if result.rowcount == 0:
            print("Test Mrpup-5 result: PASS"+'\n')
        else: 
            # MySQL to count total rows in both hop and mrpup
            query = ('''
                    SELECT hop.{col1} AS hop_bene_cnt, 
                           V2.cnt_bene_id_sum AS mrpup2_bene_cnt,
                           {rfr_physn_npi},
                           {prf_physn_npi}
                    FROM {db1}.{t1} AS hop
                    JOIN (SELECT
                            SUM(mrpup.{col2}) AS cnt_bene_id_sum,
                            mrpup.{rfr_physn_npi},
                            mrpup.{prf_physn_npi}
                          FROM {db2}.{t2} AS mrpup
                          GROUP BY mrpup.{prf_physn_npi}, mrpup.{rfr_physn_npi}
                          ) AS V2
                    ON hop.{npi} = V2.{rfr_physn_npi} AND hop.{npi_dest} = V2.{prf_physn_npi}; 
            '''.format(db1 = db_hop, db2 = db_mrpup, \
                        t1 = table_hop, t2 = table_mrpup, \
                        npi = npi, npi_dest = npi_dest, \
                        rfr_physn_npi = rfr_physn_npi, prf_physn_npi = prf_physn_npi,\
                        col1 = hop_col, col2 = mrpup_col))

            cur.execute(query, params=None, multi=True)
            for result in cur.execute(query, multi=True):
                result.fetchall()                         
                print("Number of rows in both hop and mrpupis: {}".format(result.rowcount) +'\n')
                print("Mrpup-5 test result: FAIL"+'\n')
            
        cur.close()
        connection.close()

## Test Example

SELECT
  hop.patient_count  AS hop_bene_cnt,
  V2.cnt_bene_id_sum AS mrpup2_bene_cnt,
  rfr_physn_npi,
  prf_physn_npi
FROM `_amy`.test_hop_undirected_good AS hop
  JOIN (SELECT
          SUM(mrpup.cnt_bene_id) AS cnt_bene_id_sum,
          rfr_physn_npi,
          prf_physn_npi
        FROM `_amy`.test_mrpup_good AS mrpup
        GROUP BY mrpup.prf_physn_npi, mrpup.rfr_physn_npi
       ) AS V2
    ON hop.npi = V2.rfr_physn_npi AND hop.npi_dest = V2.prf_physn_npi
WHERE hop.patient_count < V2.cnt_bene_id_sum;


SELECT
  hop.patient_count  AS hop_bene_cnt,
  V2.cnt_bene_id_sum AS mrpup2_bene_cnt,
  rfr_physn_npi,
  prf_physn_npi
FROM `_amy`.test_hop_undirected_good AS hop
  JOIN (SELECT
          SUM(mrpup.cnt_bene_id) AS cnt_bene_id_sum,
          rfr_physn_npi,
          prf_physn_npi
        FROM `_amy`.test_mrpup_bad AS mrpup
        GROUP BY mrpup.prf_physn_npi, mrpup.rfr_physn_npi
       ) AS V2
    ON hop.npi = V2.rfr_physn_npi AND hop.npi_dest = V2.prf_physn_npi
WHERE hop.patient_count < V2.cnt_bene_id_sum;

In [13]:
mrpup_5('_amy', 'test_hop_undirected_good', 'npi', 'npi_dest','patient_count', \
        '_amy', 'test_mrpup_bad', 'rfr_physn_npi', 'prf_physn_npi','cnt_bene_id')

Test Mrpup-5 statement: 'SELECT hop.patient_count AS hop_bene_cnt, 
                       V2.cnt_bene_id_sum AS mrpup2_bene_cnt,
                       rfr_physn_npi,
                       prf_physn_npi
                FROM _amy.test_hop_undirected_good AS hop
                JOIN (SELECT
                        SUM(mrpup.cnt_bene_id) AS cnt_bene_id_sum,
                        mrpup.rfr_physn_npi,
                        mrpup.prf_physn_npi
                      FROM _amy.test_mrpup_bad AS mrpup
                      GROUP BY mrpup.prf_physn_npi, mrpup.rfr_physn_npi
                      ) AS V2
                ON hop.npi = V2.rfr_physn_npi AND hop.npi_dest = V2.prf_physn_npi
                WHERE hop.patient_count < V2.cnt_bene_id_sum':

Number of rows fail the test is: 22

Number of rows in both hop and mrpupis: 22

Mrpup-5 test result: FAIL



In [ ]:
mrpup_5('old_npi_hop', 'hop_teaming_same_day_merged_RQ17', 'from_npi', 'to_npi','patient_count', \
        'mrpup_carr', 'mrpup_two_npi_rfr_prf_RQ17', 'rfr_physn_npi', 'prf_physn_npi','cnt_bene_id')